### 手動function的呼叫
- enable_automatic_function_calling=False,預設值是false,代表模型不會自動呼叫,可以依靠手動呼叫,意思是寫程式自已呼叫
- 這樣的寫法就可以取得function傳出來的值

In [1]:
import os
import google.generativeai as genai

def find_movies(description:str, location:str = "") -> list[str]:
    """find movie titles currently playing in theaters based on any description, genre, title words, etc.

    Args:
        description: Any kind of description including category or genre, title words, attributes, etc.
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
    """

    return ["Barbie", "Oppenheimer"]

def find_theaters(location: str, movie: str = ""):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ["Googleplex 16", "Android Theatre"]

def get_showtimes(location: str, movie: str, theater: str, date: str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    return ["10:00", "11:00"]

functions = {
    "find_movies": find_movies,
    "find_theaters": find_theaters,
    "get_showtimes": get_showtimes,
}

genai.configure(api_key=os.environ['GEMINI_API_KEY'])

model = genai.GenerativeModel(
    model_name = 'gemini-2.0-flash-exp',
    tools = functions.values()
)

response = model.generate_content(
    "Which theaters in Mountain View show the Barbie movie?"
)
##不可以使用response.text,因為沒有自動呼叫
response.candidates[0].content.parts #檢查parts內的值

[function_call {
  name: "find_theaters"
  args {
    fields {
      key: "movie"
      value {
        string_value: "Barbie"
      }
    }
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
  }
}
]

### 利用part的值,手動呼叫,因為有function名稱,還有引數名稱和引數值
- 程式碼得到傳出來的值

In [2]:
import os
import google.generativeai as genai

def find_movies(description:str, location:str = "") -> list[str]:
    """find movie titles currently playing in theaters based on any description, genre, title words, etc.

    Args:
        description: Any kind of description including category or genre, title words, attributes, etc.
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
    """

    return ["Barbie", "Oppenheimer"]

def find_theaters(location: str, movie: str = ""):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ["Googleplex 16", "Android Theatre"]

def get_showtimes(location: str, movie: str, theater: str, date: str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    return ["10:00", "11:00"]

functions = {
    "find_movies": find_movies,
    "find_theaters": find_theaters,
    "get_showtimes": get_showtimes,
}

genai.configure(api_key=os.environ['GEMINI_API_KEY'])

model = genai.GenerativeModel(
    model_name = 'gemini-2.0-flash-exp',
    tools = functions.values()
)

response = model.generate_content(
    "Which theaters in Mountain View show the Barbie movie?"
)
##不可以使用response.text,因為沒有自動呼叫
part = response.candidates[0].content.parts[0]

def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)

if part.function_call:
    result = call_function(part.function_call, functions)

print(result)

['Googleplex 16', 'Android Theatre']
